<a href="https://colab.research.google.com/github/Gauravhulmukh/Chatbot-Development-for-Regional-language-using-Artificial-Intelligence/blob/master/chatbotmodelcomplete1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
nltk.download('punkt')#findout why we need punkt for word tokenziation? 
#Stemmer=just simple the word:helper=help
#it has two type porter and Lancaster


# things we need for Tensorflow
import numpy as np
!pip install tflearn
import tflearn
import tensorflow as tf
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Instructions for updating:
Colocations handled automatically by placer.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

# import our chat-bot intents file
import json
with open('/content/drive/My Drive/intents.json') as json_data:
    intents = json.load(json_data)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
#simply creating array
words = []
classes = []
documents = []
ignore_words = ['?']#it will ignore  question MARK from pattern

#create loop intents>patterns 
for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        w = nltk.word_tokenize(pattern) #seperate words:hi there=hi,there
        words.extend(w)
        #print(words) #it will add seperated words from each pattern into words array 
        documents.append((w, intent['tag']))
        #print(documents) #it is set of words(tokenize_pattern) & tag
        
        if intent['tag'] not in classes:
          #print(classes)
          classes.append(intent['tag'])
#print(classes)
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
#print(words)
words = sorted(list(set(words)))
#print(words)
classes = sorted(list(set(classes)))
#create classes array which contain all the tags in alphabhetical order

print (len(documents), documents)
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)



45 [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'), (['Give', 'me', 'a', 'list', 'of', 'drugs', 'causing', '

In [0]:
#creating arrays for training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
   # print(len(pattern_words),pattern_words)
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    
    
    for w in words:
      #print(w,pattern_words)
      bag.append(1) if w in pattern_words else bag.append(0)
      #print(len(bag),bag) #it is comparing alphabhetical words array & non-alphabhetical stem words of document
  #match 1 else 0
        
    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    #print(output_row)
    output_row[classes.index(doc[1])] = 1
    #print(len(output_row),output_row) #it is comparing classes array & doc[1] array

    #adding bag with 1&0 and
    training.append([bag, output_row])
    #print(len(bag))
    #print(bag)
    #print(len(output_row))
    #print(output_row)
    #print(len(training),training)

    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)#findout meaning of it--------------------------
#print(training) #it is printing 1 on 39th and 78 for matching hi,ther first pattern of pattern dataset 


# create train and test lists
train_x = list(training[:,0])#it contain bag array of 0&1 #contain 88 unique stemmed words
train_y = list(training[:,1])#it contain  classes array of 9 in 0&1 #contain 9 classes

#print(len(train_x),train_x)
#print(train_y)
#refer this https://sourcedexter.com/tensorflow-text-classification-python/
#print(training)


In [6]:
#Clears the default graph stack and resets the global default graph.
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
#print(net) Tensor("InputData/X:0", shape=(?, 88), dtype=float32)

net = tflearn.fully_connected(net, 8)
#print(net) Tensor("FullyConnected/BiasAdd:0", shape=(?, 8), dtype=float32)

net = tflearn.fully_connected(net, 8)
#print(net)  Tensor("FullyConnected_1/BiasAdd:0", shape=(?, 8), dtype=float32)

net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
#print(net)  Tensor("FullyConnected_2/Softmax:0", shape=(?, 9), dtype=float32)

net = tflearn.regression(net)
#print(net)  #Tensor("FullyConnected_2/Softmax:0", shape=(?, 9), dtype=float32)


# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_chatbot_redsamurai_medical_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('chatbot_redsamurai_medical_model.tflearn')

#print(model)
#model.load('chatbot_invoicing_model.tflearn')


Training Step: 5999  | total loss: 0.92189 | time: 0.024s
| Adam | epoch: 1000 | loss: 0.92189 - acc: 0.8836 -- iter: 40/45
Training Step: 6000  | total loss: 0.83399 | time: 0.029s
| Adam | epoch: 1000 | loss: 0.83399 - acc: 0.9057 -- iter: 45/45
--
INFO:tensorflow:/content/chatbot_redsamurai_medical_model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [0]:
#model.load('chatbot_invoicing_model.tflearn')
#print(model_path)

def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    #print(sentence_words) # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words
    
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    
    sentence_words = clean_up_sentence(sentence)
    print(sentence_words)
    # bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))
    

In [38]:
p = bow("Helloer keted", words)#p = bow("I'm looking for pharmacy", words)
print (len(p),p)
#p = bow("thanks", words)#q = bow("'s hi gaurav", words)#print(words)#print (len(q),q)
print (classes)
print(model.predict([p]))

['hello', 'ket']
found in bag: hello
82 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]
['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
[[1.0512230e-04 2.3745429e-02 9.2081522e-05 2.8300572e-03 9.2329246e-01
  1.8170330e-04 1.3831171e-02 7.1695903e-03 2.8752375e-02]]


In [0]:
print(model.predict([p]))

[[7.9038600e-04 1.6175923e-03 8.0226488e-02 8.2394434e-03 4.9419859e-03
  3.6656728e-01 7.7968165e-05 4.9243414e-01 4.5104727e-02]]


In [0]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "chatbot_redsamurai_medical_training_data", "wb" ) )
#'wb' means 'write binary'
#data = pickle.load( open( "chatbot_redsamurai_medical_training_data", "rb" ) )
#print(data)